In [1]:
import logging
import json
import torch
from types import SimpleNamespace
from vilbert.vilbert import VILBertActionGrounding, BertConfig, VILBertActionSelection
from pytorch_transformers.tokenization_bert import BertTokenizer
from pytorch_transformers.optimization import AdamW, WarmupLinearSchedule
import torch.distributed as dist
from VLN_config import config as args
import random
import pandas as pd
from data.dataLoaderActSelection import DataLoader

In [2]:
import sys
import os
import torch
import yaml

import numpy as np
import matplotlib.pyplot as plt
import PIL

from PIL import Image
import cv2
import argparse
import glob
import pdb

import torchvision.models as models
import torchvision.transforms as transforms

from faster_rcnn import feature_extractor_new as f_extractor
from faster_rcnn.feature_extractor_new import featureExtractor, visualize_tensor
#%matplotlib inline  

In [3]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)
print(args)

namespace(adam_epsilon=1e-08, baseline=False, bert_model='bert-base-uncased', best_features=5, clean_train_sets=True, clip_size=3, config_file='config/bert_base_6layer_6conect.json', distributed=False, do_lower_case=True, dynamic_attention=False, epochs=100, from_pretrained='save/action_grounding/best_val.bin', gradient_accumulation_steps=1, img_weight=1, in_memory=False, learning_rate=0.0001, local_rank=-1, max_temporal_memory_buffer=3, mean_layer=False, num_key_frames=2, num_train_epochs=10.0, num_workers=0, objective=1, predict_feature=False, save_name='', seed=42, split='mteval', start_epoch=0, task_specific_tokens=True, tasks='1', threshold_similarity=0.7, track_temporal_features=True, train_batch_size=4, use_tensorboard=True, visual_target=0, warmup_proportion=0.1, without_coattention=False)


In [4]:
tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

08/03/2020 13:13:16 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/mikel/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [5]:
config = BertConfig.from_json_file(args.config_file)
bert_weight_name = json.load(
    open("config/" + args.bert_model + "_weight_name.json", "r")
)

tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

config.track_temporal_features = args.track_temporal_features
config.mean_layer = args.mean_layer
config.max_temporal_memory_buffer = args.max_temporal_memory_buffer
config.visualization = True

#The path of the finetuned ActionGrounding weights
args.from_pretrained = "./save/action_selection/best_train_vilberActionSelection.bin"

print(args.from_pretrained)
model = VILBertActionSelection.from_pretrained(
    args.from_pretrained, config=config, default_gpu=True
)
model.eval()

08/03/2020 13:13:17 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/mikel/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
08/03/2020 13:13:17 - INFO - vilbert.utils -   loading weights file ./save/action_selection/best_train_vilberActionSelection.bin
./save/action_selection/best_train_vilberActionSelection.bin


tLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
          )
          (output): BertOutput(
            (dense): Linear(in_features=3072, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (v_layer): Mod

In [6]:
# load data
frcnn_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
data_loader = DataLoader("data/json_data.json", frcnn_model, save_or_not = False)
path = 'data/DataLoaderActSelection.pt'
data_loaded = data_loader.load_dataloader(path)

In [8]:
features_masked, pos_enc, spatial, image_mask, tokenized_text, masked_text, masked_lm_token, input_mask, segment_ids, co_attention_mask, infos, masked_img_labels  = data_loaded

ValueError: not enough values to unpack (expected 12, got 10)

In [7]:
features, pos_enc, spatial, image_mask, tokenized_text, input_mask, segment_ids, co_attention_mask, infos, action_targets = data_loaded


In [8]:
#Forward the model with one input example
model.eval()
pred_action_train, att_train = model(input_ids = tokenized_text[:5].cpu(),
                                image_feat = features[:5].cpu(), # Linear(2048*config.max_temporal_memory_buffer, 2048)
                                image_loc = spatial[:5].cpu(),  #Linear(in_features=5, out_features=1024, bias=True)
                                image_pos_input = pos_enc[:5].cpu(),   #Linear(7, 2048)/(6, 2048)
                                token_type_ids = segment_ids[:5].cpu(), 
                                attention_mask = input_mask[:5].cpu(), 
                                image_attention_mask = image_mask[:5].cpu(),
                                output_all_attention_masks=True)

## Action prediction and attention

In [11]:
data_loader = DataLoader("data/json_data.json", frcnn_model, save_or_not = False)
i = 0
id_action_masked = torch.argmax(pred_action_train[i]).item()
print("predicted token" , id_action_masked)
print("gt token : ", action_targets[i].item())
predicted_masked_action = data_loader.id_to_action(id_action_masked)
print("Prediction: ", predicted_masked_action)
print("GT: ",data_loader.id_to_action(action_targets[i].item()))
from sklearn.metrics import accuracy_score as accuracy
acc = accuracy(action_targets[:5].numpy(), torch.argmax(pred_action_train, dim=1).numpy())
print("The accuracy over train data set: ", acc*100)

predicted token 4
gt token :  4
Prediction:  LookDown
GT:  LookDown
The accuracy over train data set:  40.0


In [ ]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

In [ ]:
import torch

def format_attention(attention):
    squeezed = []
    for layer_attention in attention:
        # 1 x num_heads x seq_len x seq_len
        if len(layer_attention.shape) != 4:
            raise ValueError("The attention tensor does not have the correct number of dimensions. Make sure you set "
                             "output_attentions=True when initializing your model.")
        squeezed.append(layer_attention.squeeze(0))
    # num_layers x num_heads x seq_len x seq_len
    return torch.stack(squeezed)

def format_special_chars(tokens):
    return [t.replace('Ġ', ' ').replace('▁', ' ').replace('</w>', '') for t in tokens]

In [ ]:
import json
from IPython.core.display import display, HTML, Javascript
import os



def head_view(attention, tokens, tokens_l, sentence_b_start = None, prettify_tokens=False):
    """Render head view
        Args:
            attention: list of ``torch.FloatTensor``(one for each layer) of shape
                ``(batch_size(must be 1), num_heads, sequence_length, sequence_length)``
            tokens: list of tokens
            sentence_b_index: index of first wordpiece in sentence B if input text is sentence pair (optional)
            prettify_tokens: indicates whether to remove special characters in wordpieces, e.g. Ġ
    """

    if sentence_b_start is not None:
        vis_html = """
        <span style="user-select:none">
            Layer: <select id="layer"></select>
            Attention: <select id="filter">
              <option value="all">All</option>
              <option value="aa">Sentence A -> Sentence A</option>
              <option value="ab">Sentence A -> Sentence B</option>
              <option value="ba">Sentence B -> Sentence A</option>
              <option value="bb">Sentence B -> Sentence B</option>
            </select>
            </span>
        <div id='vis'></div>
        """
    else:
        vis_html = """
              <span style="user-select:none">
                Layer: <select id="layer"></select>
              </span>
              <div id='vis'></div> 
            """

    display(HTML(vis_html))
    __location__ = os.path.realpath(
        os.path.join(os.getcwd()))#, os.path.dirname(__file__)))
    vis_js = open(os.path.join(__location__, 'head_view.js')).read()

    if prettify_tokens:
        tokens = format_special_chars(tokens)

    attn = format_attention(attention)
    attn_data = {
        'all': {
            'attn': attn.tolist(),
            'left_text': tokens_l,
            'right_text': tokens
        }
    }
    
    params = {
        'attention': attn_data,
        'default_filter': "all"
    }
    attn_seq_len = len(attn_data['all']['attn'][0][0])
    print(attn_seq_len)
    if attn_seq_len != len(tokens):
        pass
        #raise ValueError(f"Attention has {attn_seq_len} positions, while number of tokens is {len(tokens)}")

    display(Javascript('window.params = %s' % json.dumps(params)))
    display(Javascript(vis_js))

In [ ]:
classes = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
# (all_attention_mask_t, all_attnetion_mask_v, all_attention_mask_c) = att_train
def preprocess_att_viz(data_idx, att_idx):
    global att_train, infos, classes
    att_list = []
    for i in range(len(att_train[att_idx])):
        if att_idx == 2:
            att_list.append(att_train[att_idx][i]["attn2"][data_idx].unsqueeze(0))
        else:
            att_list.append(att_train[att_idx][i]["attn"][data_idx].unsqueeze(0))
    input_id_list = masked_text[data_idx].tolist()
    tokens = tokenizer.convert_ids_to_tokens(input_id_list) 
#     img_reg = [1]*att_list[att_idx].shape[2] + [0]*att_list[att_idx].shape[-1]
    img_reg = [classes[info] for info in infos[0][0]["objects"].tolist()] # + ['PAD']*abs(att_list[att_idx].shape[-1] - infos[data_idx]["objects"].shape[0])  
    if att_idx==0:
        return att_list, tokens, tokens
    if att_idx==1:
        return att_list, img_reg, img_reg
    else:
        return att_list, tokens, img_reg

In [ ]:
att_list, token1, token2= preprocess_att_viz(2, 0)
head_view(att_list, token1, token1)

## Image prediction and attention

In [ ]:
i=5
#print(tokenizer.convert_ids_to_tokens(masked_text[i].tolist()))
#To set which word is masked
for j in range(len(masked_img_labels[i])):
    if masked_img_labels[i][j].item()!=-1:
        break
id_word_masked = torch.argmax(pred_v_train[i,j,:]).item()
print("predicted token" , id_word_masked)
print("gt token : ", masked_img_labels[i][j].item())
print("Prediction: ", classes[id_word_masked])
print("GT: ", classes[masked_img_labels[i][j].item()])

In [ ]:
att_list, token1, token2= preprocess_att_viz(0, 1)
head_view(att_list, token2, token2)

### Image and Text prediction co-attention

In [ ]:
att_list, token1, token2= preprocess_att_viz(0, 2)
head_view(att_list, token1, token2)

In [ ]:
print(infos[0][0])
visualize_tensor()